Model Evaluation

In [1]:
import os
os.chdir("../")
%pwd

'd:\\cboobs\\textSummarizer'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvalConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_path: Path

In [3]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

class ConfigManager:
    def __init__(
            self, config_path=CONFIG_FILE_PATH, 
            params_path=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_directories([self.config.artifacts_root])

    def get_model_eval_config(self) -> ModelEvalConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_eval_config = ModelEvalConfig(
            root_dir = config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_path=config.metric_file_path
        )

        return model_eval_config

In [6]:
pip install evaluate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import pandas as pd
from datasets import load_dataset, load_from_disk
from evaluate import load
from tqdm import tqdm


class ModelEvaluation:
    def __init__(self, config: ModelEvalConfig):
        self.config = config


    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    
    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="article", 
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            # Finally, we decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score


    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)


        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
  
        rouge_metric = load('rouge')

        score = self.calculate_metric_on_test_ds(
        dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
            )

        rouge_dict = {}
        for metric_name in rouge_names:
            if metric_name in score:
                rouge_dict[metric_name] = score[metric_name]
            else:
                print(f"Warning: {metric_name} not found in score")

        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_file_path, index=False)

In [6]:
try:
    config = ConfigManager()
    model_eval_config = config.get_model_eval_config()
    model_evaluator = ModelEvaluation(config=model_eval_config)
    model_evaluator.evaluate()
except Exception as e:
    raise e

[2025-09-19 20:01:55,916: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2025-09-19 20:01:55,923: INFO: common: yaml file: params.yaml loaded successfully:]
[2025-09-19 20:01:55,925: INFO: common: Created directory at: artifacts:]
[2025-09-19 20:01:55,927: INFO: common: Created directory at: artifacts/model_evaluation:]


100%|██████████| 5/5 [50:27<00:00, 605.50s/it]


[2025-09-19 20:52:27,724: INFO: rouge_scorer: Using default tokenizer.:]
